In [35]:
import pylab as plt

In [38]:
from healpyTools import rotate_healpix_map as rhm

In [59]:
import numpy as np
import healpy as hp
import zm_tools as zt
import instrumental_response_tensors as irt

In [146]:
nside = 64
npix = hp.nside2npix(nside)
rdindx = np.arange(npix) # healpix pixel index for Ra/Dec coords
codec,ra = hp.pix2ang(nside,rdindx)
# codec = np.pi - dec
dec = np.pi - codec

In [155]:
# A constant sky in the za,ha basis and (ha,dec) coordinates


    
## A flat sky    
if True:
    I = 10. * np.ones(npix)
    Q = 0. * np.ones(npix)
    U = 0. * np.ones(npix)
    V = 0. * np.ones(npix)



# get an active rotation matrix to the z'-axis through (dec,ha)=(-26,0)
dec0 = -26. # degrees
ha0 = 0. # degrees
codec0 = 90. - dec0 # degrees

Rinv = hp.Rotator(rot=[0,-codec0,0])
R = R.get_inverse()
rotation_matrix = np.asarray(Rinv.mat)

## A single point(ish) source at zenith:
if False:
    cele_pole_idx = [0,1,2,3]
    cp_theta,cp_phi = hp.pix2ang(nside,cele_pole_idx)
    z_theta,z_phi = R(cp_theta,cp_phi)
    z_idx = hp.ang2pix(nside,z_theta,z_phi)
    I = np.zeros(npix)
    I[z_idx] = 10.
    Q = 0. * np.ones(npix)
    U = 0. * np.ones(npix)
    V = 0. * np.ones(npix)

sphr_basis_rotation = zt.get_sphr_rotation_matrix(codec,ra,rotation_matrix)

In [62]:
# get the HERA Jones matrix
jones_data = irt.get_hera_jones_healpixellized_lores() # healpix maps

jones_ae_ae = jones_data['jones'] # jones.shape = (2,2,npix). 

pixels with no hits (0,)
pixels with no hits (0,)
pixels with no hits (0,)
pixels with no hits (0,)
pixels with no hits (0,)
pixels with no hits (0,)
pixels with no hits (0,)
pixels with no hits (0,)


In [156]:
# We define these maps to be in local Az/El coordinates, so lets change that:

window_idx = np.where(codec < np.pi /2.) # treating codec as the theta coordinate
beam_window = np.zeros(npix)
beam_window[window_idx] = 1.

theta0,phi0 = Rinv(codec,ra)
ae0index = hp.ang2pix(nside,theta0,phi0)

jones_hd_ae = jones_ae_ae[:,:,ae0index] 
beam_window0 = beam_window[ae0index]

beam_window0_arr = np.repeat(beam_window0,4).reshape(npix,2,2).transpose((1,2,0))

jones_masked_hd_ae = jones_hd_ae * beam_window0_arr


# This is the Jones matrix that maps from the sky in the Az/El basis to the beam-weighted sky in the instrumental basis at time t = ha+ra = 0. 
# ((This is probably not quite right. The point is that we're going to rotate the scalar components to simulate the rotation of the instrument under the sky.))
# However, the scalar components are to be considered functions of Ha/Dec.

# Maybe this is a Class? Then it can have attributes like "Coordinate system" and "Vector basis" and methods to change between them, 'n stuff.

jones_hd_hd = np.einsum('ab...,bc...->ac...',jones_masked_hd_ae,sphr_basis_rotation)

jonesTc_hd_hd = np.conjugate(jones_hd_hd.transpose(1,0,2))

sky_hd_hd = np.array([[I+Q,U -1.j *V],[U+1.j*V,I-Q]])

## Lets see about some propagators for one baseline

u,v = 50.,50.

s = np.array(hp.pix2vec(nside,np.arange(npix)))
b = np.outer(np.array([u,v]),np.ones(npix))

b_dot_s = np.einsum('a...,a...',s[:2,:],b)
nu = 150. * 10.**6. # Hz
c = 299792458. # m/s

propagator_ae = np.exp(1.j * nu * b_dot_s / c)
propagator_hd = propagator_ae[ae0index]
propagator_hd_arr = np.repeat(propagator_hd,4).reshape(npix,2,2).transpose((1,2,0))

temp = np.einsum('ab...,bc...->ac...',sky_hd_hd,jonesTc_hd_hd)
holographic_sky_hd_instr = np.einsum('ab...,bc...->ac...',jones_hd_hd,temp)

vis_150_0 = np.sum(holographic_sky_hd_instr * propagator_hd_arr,axis=2)

In [157]:
print vis_150_0

[[ 12.42408694-7.78386136j  -8.64195803-2.05735143j]
 [ 53.25479817-3.9073314j   13.75474128-9.00533988j]]


In [153]:
    cele_pole_idx = [0,1,2,3]
    cp_theta,cp_phi = hp.pix2ang(nside,cele_pole_idx)
    z_theta,z_phi = R(cp_theta,cp_phi)
    z_idx = hp.ang2pix(nside,z_theta,z_phi)

In [154]:
print beam_window0[z_idx]
print beam_window[[0,1,2,3]]

[ 1.  1.  1.  1.]
[ 1.  1.  1.  1.]


In [121]:
print jones_hd_hd[:,:,z_idx].transpose(2,0,1)

[[[ 0.+0.j  0.+0.j]
  [ 0.+0.j  0.+0.j]]

 [[ 0.+0.j  0.+0.j]
  [ 0.+0.j  0.+0.j]]

 [[ 0.+0.j  0.+0.j]
  [ 0.+0.j  0.+0.j]]

 [[ 0.+0.j  0.+0.j]
  [ 0.+0.j  0.+0.j]]]


In [120]:
print jones_ae_ae[:,:,[0,1,2,3]].transpose(2,0,1)

[[[ 0.04759674+0.70275372j -0.52763882+0.46709615j]
  [ 0.04647283+0.70213759j  0.04083421+0.70282716j]]

 [[-0.52644088+0.468525j   -0.53189902+0.46289208j]
  [ 0.04101920+0.70249987j -0.53150743+0.46164769j]]

 [[-0.53134524+0.4629162j   0.04157543+0.70392665j]
  [-0.53134984+0.46180844j -0.52828889+0.4659333j ]]

 [[ 0.04189387+0.70313873j  0.04577451+0.70318832j]
  [-0.52661020+0.4672289j   0.04446058+0.70297312j]]]


In [86]:
hp.mollview(I)
plt.show()

In [150]:
hp.mollview(beam_window)
hp.mollview(beam_window0_arr[0,1,:])
plt.show()

In [92]:
hp.orthview(np.arctan2(propagator_hd.real,propagator_hd.imag))
plt.show()

In [93]:
Rinv = R.get_inverse()

In [94]:
print Rinv.mat
print R.mat

[[-0.43837115  0.         -0.89879405]
 [ 0.          1.          0.        ]
 [ 0.89879405  0.         -0.43837115]]
[[-0.43837115  0.          0.89879405]
 [ 0.          1.          0.        ]
 [-0.89879405  0.         -0.43837115]]


In [151]:
dirx = np.abs(jones_hd_hd[0,0,:])**2. + np.abs(jones_hd_hd[0,1,:])**2.

In [152]:
hp.mollview(dirx)
plt.show()

In [110]:
np.where(dirx == dirx.max())[0][0]

35455

In [112]:
z_idx

array([35327, 35584, 35584, 35328])

In [119]:
testI = np.zeros(npix)
testI[z_idx] = 10.
hp.mollview(dirx)
hp.mollview(testI)
plt.show()

In [130]:
sphr_basis_rotation.shape

(2, 2, 49152)

In [145]:
hp.orthview(rhm(sphr_basis_rotation[0,1,:],rot=[180,90]))
plt.show()